In [591]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from tensorflow.python import keras
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout


In [592]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [593]:
# cargamos nuestro dataset
train_data_path = '/content/drive/MyDrive/Colab Notebooks/train.csv'
test_data_path = '/content/drive/MyDrive/Colab Notebooks/test.csv'

In [594]:
# desplegamos nuestro dataset de entrenamiento
dtrain = pd.read_csv(train_data_path, index_col = "PassengerId")
dtrain.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [595]:
# desplegamos nuestro dataset de prueba
tdata = pd.read_csv(test_data_path)
tdata.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [596]:
# las columnas que tendran poco impacto en el dataset son, Name, Cabin, Ticket
# variables categoricas Name, Sex, Cabin, Embarked
encoder = preprocessing.LabelEncoder()

In [597]:
# hacemos la lista de las caracteristicas a las que queremos hacer encode
cat_features = ['Sex', 'Embarked']

In [598]:
#convertimos a string los valores de la columna Embarked
dtrain['Embarked']=dtrain['Embarked'].astype(str)

In [599]:
#codificamos el conjunto de datos de entrenamiento
encoded_train = dtrain[cat_features].apply(encoder.fit_transform)
encoded_test = tdata[cat_features].apply(encoder.fit_transform)

In [600]:
# juntamos todas nuestras columnas numericas excepto ticket.
num_features = ["Survived","Pclass","Age","SibSp","Parch","Fare"]
test_features= ["Pclass","Age","SibSp","Parch","Fare"]

In [601]:
# unimos los datos codificados en nuestro conjunto de test y train, cuando lo hagamos
#solo tomamos las columnas que queramos (menos Name, Cabin, y Ticket)
training_data = dtrain[num_features].join(encoded_train)
test_data = tdata[test_features].join(encoded_test)
training_data

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
PassengerId,,,,,,,,
1,0,3,22.0,1,0,7.2500,1,2
2,1,1,38.0,1,0,71.2833,0,0
3,1,3,26.0,0,0,7.9250,0,2
4,1,1,35.0,1,0,53.1000,0,2
5,0,3,35.0,0,0,8.0500,1,2
...,...,...,...,...,...,...,...,...
887,0,2,27.0,0,0,13.0000,1,2
888,1,1,19.0,0,0,30.0000,0,2
889,0,3,NaN,1,2,23.4500,0,2


In [602]:
test_data.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3,34.5,0,0,7.8292,1,1
1,3,47.0,1,0,7.0000,0,2
2,2,62.0,0,0,9.6875,1,1
3,3,27.0,0,0,8.6625,1,2
4,3,22.0,1,1,12.2875,0,2


In [603]:
# verificamos nuestros valores null
training_data.isnull().sum()

Survived      0
Pclass        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex           0
Embarked      0
dtype: int64

In [604]:
# lo anterior se aplica los datos de prueba
test_data.isnull().sum()

Pclass       0
Age         86
SibSp        0
Parch        0
Fare         1
Sex          0
Embarked     0
dtype: int64

In [605]:
# usaremos la imputacion para llenar nuestros datos con la media
my_imputer = SimpleImputer()
imputed_train = pd.DataFrame(my_imputer.fit_transform(training_data))
imputed_test_data = pd.DataFrame(my_imputer.fit_transform(test_data))

In [606]:
imputed_train.head()

,0,1,2,3,4,5,6,7
0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [607]:
imputed_train.columns = training_data.columns
imputed_test_data.columns = test_data.columns


In [608]:
imputed_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,0.0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,0.0,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [609]:
# ahora podemos dividir nuestros datos.
# X son nuestros predictores
# y lo que queremos predecir ("Survived")

X = imputed_train.drop('Survived', axis = 1)
y = imputed_train['Survived']

In [610]:
# Desplegando X
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
0,3.0,22.0,1.0,0.0,7.2500,1.0,2.0
1,1.0,38.0,1.0,0.0,71.2833,0.0,0.0
2,3.0,26.0,0.0,0.0,7.9250,0.0,2.0
3,1.0,35.0,1.0,0.0,53.1000,0.0,2.0
4,3.0,35.0,0.0,0.0,8.0500,1.0,2.0


In [611]:
# Desplegando Y
y.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

In [612]:
# establecemos nuestra semilla aleatoria
tf.random.set_seed(42)

In [613]:
clasificador = Sequential()
# Primera Capa
clasificador.add(Dense(7, activation ='relu', input_shape=(7,),kernel_initializer='random_normal')) 
clasificador.add(Dropout(0.2))

# Segunda Capa con 50 neuronas y una activacion relu
clasificador.add(Dense(50, activation = "relu",kernel_initializer='random_normal'))
clasificador.add(Dropout(0.2))

# Tercera Capa con 50 neuronas y una activacion relu
clasificador.add(Dense(50, activation = "relu",kernel_initializer='random_normal'))
clasificador.add(Dropout(0.2))

# Capa de Salida
# utilizamos la activacion sigmoid, esta devolvera una probabilidad para cada resultado y elegiremos la probabilidad mas alta como nuestra prediccion
clasificador.add(Dense(2, activation = "sigmoid",kernel_initializer='random_normal'))


In [614]:
# parametros de optimizacion
clasificador.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer = "SGD", metrics = ['accuracy'])

In [615]:
# Entrenando nuestro modelo
clasificador.fit(X, y, batch_size=1, epochs=500, validation_split = 0.2, verbose = 1)

Epoch 1/500
712/712 [==============================] - 2s 2ms/step - loss: 0.6603 - accuracy: 0.6096 - val_loss: 0.6143 - val_accuracy: 0.6425
Epoch 2/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6557 - accuracy: 0.6081 - val_loss: 0.6564 - val_accuracy: 0.6425
Epoch 3/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6394 - accuracy: 0.6264 - val_loss: 0.6330 - val_accuracy: 0.6760
Epoch 4/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6498 - accuracy: 0.6138 - val_loss: 0.5831 - val_accuracy: 0.6425
Epoch 5/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6507 - accuracy: 0.6124 - val_loss: 0.6203 - val_accuracy: 0.6425
Epoch 6/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6467 - accuracy: 0.6180 - val_loss: 0.5922 - val_accuracy: 0.6425
Epoch 7/500
712/712 [==============================] - 1s 2ms/step - loss: 0.6555 - accuracy: 0.6067 - val_loss: 0.6151 - val_accuracy: 0.6425

In [616]:
# usamos nuestro modelo para predecir
path = '/content/drive/MyDrive/Colab Notebooks/'
clasificador.save(path + 'Modelo_V1.h5')

In [617]:
new_model = keras.models.load_model(path + 'Modelo_V1.h5')

In [618]:
y_preds = new_model.predict(imputed_test_data)
y_preds

array([[0.6361052 , 0.3543626 ],
       [0.6265121 , 0.36579382],
       [0.6630378 , 0.32236636],
       [0.6292408 , 0.3625039 ],
       [0.56353104, 0.44374555],
       [0.61183864, 0.3835003 ],
       [0.5772178 , 0.4217576 ],
       [0.57634497, 0.4289162 ],
       [0.5095564 , 0.5049968 ],
       [0.6028131 , 0.39587694],
       [0.63506246, 0.35547066],
       [0.6058281 , 0.3911231 ],
       [0.4182668 , 0.6035568 ],
       [0.63813764, 0.35266668],
       [0.4182668 , 0.6035568 ],
       [0.4182668 , 0.6035568 ],
       [0.62412596, 0.3689572 ],
       [0.61680967, 0.3777196 ],
       [0.59938926, 0.39858362],
       [0.59508973, 0.3994256 ],
       [0.4182668 , 0.6035568 ],
       [0.62179536, 0.37159833],
       [0.4182668 , 0.6035568 ],
       [0.4182668 , 0.6035568 ],
       [0.4182668 , 0.6035568 ],
       [0.65235436, 0.33548084],
       [0.4182668 , 0.6035568 ],
       [0.6186569 , 0.37548828],
       [0.58438295, 0.41682446],
       [0.6118638 , 0.38513458],
       [0.

In [619]:
y_final = np.array(clasificador.predict(imputed_test_data)).argmax(axis=1)

In [620]:
print(y_final)

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0
 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 0 0 1
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0
 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0
 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 1 0 1 0 1 0 0 1 0 0 0]


In [621]:
passenger_id=tdata["PassengerId"]
results=passenger_id.to_frame()
results["Survived"]=y_final

In [622]:
results.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [623]:
results.to_csv("ModeloFinal_V6.csv", index = False)